In [1]:
# import statements
import glob
import numpy as np 
import cv2
import pdb 
import time
from sklearn.decomposition import PCA
from sklearn.neighbors import KDTree
import sys 

In [2]:
def read_images(img_dir):
    # read files in directory and add image to list
    all_images = []
    if not img_dir.endswith("/"):
        img_dir = img_dir + "/"
    filenames = glob.glob(img_dir + "*.jpg")
    tmp_file = filenames[0]
    tmp_img = cv2.imread(tmp_file)
    img_shape = tmp_img.shape
    data_matrix = np.empty((500, img_shape[0]*img_shape[1]*img_shape[2]))
    count = 0
    for file in filenames[0:500]:
        tmp_img = cv2.imread(file)
        all_images.append(tmp_img)
        
        # d.)
        # You complete this part --------------------------------------------------->
        # You should reshape tmp_img to a vector and assign it to reshaped_img
        # reshaped_img = ??
        reshaped_img = np.array(tmp_img).flatten()
        data_matrix[count, :] = reshaped_img
        count = count + 1
    
    
    return all_images, data_matrix
        
def create_data_matrix(images):
    # reshape each matrix and add to array 
    data_matrix = []
    for image in images: 
        tmp_image = image.flatten()[np.newaxis, :]
        data_matrix.append(tmp_image)
        
        
    data_matrix = np.concatenate(data_matrix, axis=0)
    return data_matrix

def get_closest_face(data_matrix, query_face):
    # data_matrix : matrix of all images, after transformed using directions of maximal variation
    # query_face : vector of query image, after transformed using directions of maximal variation
    # returns index of closest face in data set
    kdt = KDTree(data_matrix, leaf_size=30, metric='euclidean')
    idx = kdt.query(query_face, k=1, return_distance=False) 

    return idx

In [4]:
if __name__ == "__main__":

    IMG_DIR =  '../data/img_align_celeba/'
    N_EIGEN_FACES = 15
    MAX_SLIDER_VALUE = 255

    all_images, data_matrix = read_images(IMG_DIR)
    IMG_SHAPE = all_images[0].shape

   
    # do PCA analysis
    print("Doing PCA analysis ...")
    start_time = time.time()
    

    pca = PCA(n_components=N_EIGEN_FACES)  
    # e.)
    # You complete this -------------------------------------------------------->
    # fit pca object with data matrix
    data_martrix=pca.fit_transform(data_matrix)
    

    # You complete this -------------------------------------------------------->
    # get pca mean 
    mean = np.mean(data_matrix, axis=0)
    
    # You complete this -------------------------------------------------------->
    # return list of directions of maximal variation (in order), Hint: look at pca.components
    # eigenvectors is a list of directions of maximal variation
    # eigenvectors = ??
    eigenvectors = pca.components_
    
    # we'll process this list for you
    eigenvectors = [np.asarray(eigenvectors[i]) for i in range(len(eigenvectors))]
    end_time = time.time()
    print("Done!")
    print("Duration: ", end_time - start_time)

    mean_face = mean.reshape(IMG_SHAPE)
    mean_face = np.asarray(mean_face, dtype=np.uint8)

    slider_values = []
    eigen_faces = []
    for eigenvector in eigenvectors:
        # f.)
        # You complete this ------------------------------------------------------------->
        # tmp_face is eigenvector reshaped to an image
        # complete code to reshape 
        # tmp_face = ??
        tmp_face =eigenvector.reshape(IMG_SHAPE)
        
        # we'll handle the rest for you :)
        eigen_faces.append(tmp_face)

    def make_face(*args):
        new_face = mean_face
        for i in range(N_EIGEN_FACES):
            slider_values[i] = cv2.getTrackbarPos("Weight" + str(i), "Trackbars")
            weight = slider_values[i] - MAX_SLIDER_VALUE/2
            new_face = new_face + eigen_faces[i]*weight*100
            new_face = np.maximum(np.minimum(new_face, 255),0)
            new_face = np.asarray(new_face, dtype=np.uint8)


        new_face = cv2.resize(new_face, (0,0), fx=2, fy=2)
            
        cv2.imshow("Demo face", new_face)
    #------------------------------------------------------------------------------------------
        
    cv2.namedWindow("Demo face", cv2.WINDOW_AUTOSIZE)
    output = cv2.resize(mean_face, (0,0), fx=2, fy=2)
    cv2.imshow( "Demo face", output)


    cv2.namedWindow("Trackbars", cv2.WINDOW_AUTOSIZE)

    for i in range(N_EIGEN_FACES):
        slider_values.append(int(MAX_SLIDER_VALUE/2))
        cv2.createTrackbar("Weight" + str(i), "Trackbars", int(MAX_SLIDER_VALUE/2), MAX_SLIDER_VALUE, make_face) 

    cv2.waitKey(0)
    cv2.destroyAllWindows()

Doing PCA analysis ...
Done!
Duration:  3.897240161895752
